In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

E:\New volume\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset, dtype=int, drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1 )

In [8]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]

In [9]:
dep=dataset["Purchased"]

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(indep,dep,test_size=1/3, random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [12]:
from sklearn.svm import SVC

In [14]:
from sklearn.model_selection import GridSearchCV

param_grid ={'kernel':['linear','rbf','poly','sigmoid'],
             'gamma':['auto','scale'],
             'C':[10,100,1000,2000,3000]}

grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3, cv=5, n_jobs=-1, scoring='f1_weighted')

grid.fit(X_train, y_train)




Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [15]:
re=grid.cv_results_

In [16]:
grid_predictions=grid.predict(X_test)


In [24]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)

In [25]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [26]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test, grid_predictions, average='weighted')

print("The f1_macro value for best parameter{}:",format(grid.best_params_),f1_macro)

The f1_macro value for best parameter{}: {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'} 0.9100355779243318


In [27]:
print("The Confusion Matrix:\n",cm)

The Confusion Matrix:
 [[80  5]
 [ 7 42]]


In [28]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93        85
           1       0.89      0.86      0.88        49

    accuracy                           0.91       134
   macro avg       0.91      0.90      0.90       134
weighted avg       0.91      0.91      0.91       134



In [29]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9539015606242497